# Architecture OneClassSVM
A Support Vector Machine (SVM), supervised learning method that can be use for outliers detection.  
It takes arrays (vector) and determines the values and range that makes them similar. Then it will try to detect if any new data is part of the training set or not.

In [ ]:
import junodch_utils_read_img as utils

import matplotlib.pyplot as matPlt
import numpy as np
from sklearn.svm import OneClassSVM
from sklearn.metrics import confusion_matrix
from shapely.geometry import Polygon, box

import rasterio
from rasterio import plot as rastPlt
from rasterio.mask import mask as rasterMask
from rasterio.plot import reshape_as_raster

# Data preparation
### Fetch data from file

In [ ]:
def formatDataForSVM(data):
  return data.reshape(data.shape[0], -1)

def displayResultSVM(svm, dataInput, res=64):
  MAX_ON_ROW = 20
  total = len(dataInput)
  nRow = (total // MAX_ON_ROW) + 1
  nCol = MAX_ON_ROW if total > MAX_ON_ROW else total

  score = svm.predict(formatDataForSVM(dataInput))
  matPlt.figure(figsize=(30,nRow*2))
  for i in range(0, total):
    ax = matPlt.subplot(nRow, nCol, 1+i)
    matPlt.title('T' if score[i] == 1 else 'F')
    matPlt.imshow(dataInput[i])
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
  matPlt.show()


In [ ]:
folderName = "img/Sokoto/"
pathSatellite = folderName + "Sentinel-2.tif"
pathNight = folderName + "Night_VIIRS.tif"
pathValidation = folderName + "Population_GHSL.tif"

aoi = utils.getImgBorder(pathSatellite)

# Fetch coords
dataCoords, dataRadiance = utils.getTilesCoordsPerimeter(pathNight, area=aoi)

trainMask = dataRadiance>0.2
lightCoords = dataCoords[trainMask]

print('Tiles:',dataCoords.shape[0])
print('Light Tile:',lightCoords.shape[0])


#### Fetch images

In [ ]:
with rasterio.open(pathSatellite) as f:
  trainData, _ = utils.coordsToImgsFormated(f, lightCoords, res=64)
trainDataFormated = formatDataForSVM(trainData)
print(trainData.shape)
print(trainDataFormated.shape)

### Validation
Extract the data from GHSL for the validation process

In [ ]:
with rasterio.open(pathValidation) as f:
  sampleTile, _ = rasterMask(f, [Polygon(dataCoords[1000])], crop=True) # arbitrary coordinate for a sample
print('Validation shape:',sampleTile.shape) # shape sample

print('Process validation...')
# If any of the pixels have a settlement then true else false
getValid = lambda data : [ int(250/255 < img.max()) for img in data ]
resultValid = utils.scanSatellite(pathValidation, dataCoords, getValid, batch=1000, res=sampleTile.shape[1])

## svm
Train and validate

In [ ]:
svm = OneClassSVM(verbose=False, gamma='scale', kernel='rbf', nu=0.8, tol=1e-3)
svm = svm.fit(trainDataFormated, sample_weight=dataRadiance[trainMask])

## Analyse model
Calculate all tiles prediction by the model.

In [ ]:
print('Process score...')
getScore = lambda data : [1 if i == 1 else 0 for i in svm.predict(formatDataForSVM(data))]
result = utils.scanSatellite(pathSatellite, dataCoords, getScore, batch=100, res=64)

### Confusion Matrix. 

In [ ]:
print('Process confustion matrix...')
print('total',len(result))
confusionMatrix = confusion_matrix(resultValid, result)
print(confusionMatrix)
tp = confusionMatrix[1][1]
fp = confusionMatrix[0][1]
fn = confusionMatrix[1][0]
print('f-score:',round(tp / (tp + (fp + fn)/2) * 100, 2),"%")
print('tp / fp:',round(tp / (tp + fp) * 100,2),"%")

# Display the analysed results

In [ ]:
resultImg, resultMeta = utils.mapResultOnImg(pathNight, dataCoords, result, resultValid)

fig, axs = matPlt.subplots(1,3, dpi=240)
with rasterio.open(pathSatellite) as s: utils.displayTiles([s.read()], [s.transform],axs[0])

axs[2].set_xlim(axs[0].get_xlim())
axs[2].set_ylim(axs[0].get_ylim())

with rasterio.open(pathValidation) as p: rastPlt.show(p, ax=axs[2])

axs[1].set_xlim(axs[0].get_xlim())
axs[1].set_ylim(axs[0].get_ylim())

utils.displayTiles([resultImg], [resultMeta], axs[1])

In [ ]:
img = np.copy(resultImg)
img = img[0:3,:,:]
img = img.transpose([1, 2, 0])
with rasterio.open(pathNight) as f:
  profile = f.profile
img = reshape_as_raster(img)
profile.update(count=3)
print(profile)
print(img.shape)
with rasterio.open(folderName + 'svm_result.tif', 'w', **profile) as f:
  f.write(img)

## Test

In [ ]:
indexesTest = [*np.argwhere(trainMask)[-20:].flatten(), *range(0,1), *range(1118,1127), *range(4005,4010), *range(10005,10010)]

with rasterio.open(pathSatellite) as f:
  dataTest, _ = utils.coordsToImgsFormated(f, dataCoords[indexesTest], res=64)

displayResultSVM(svm, dataTest)